# GUS Dataset Annotation with Gemini

This notebook annotates the `ethical-spectacle/gus-dataset-v1` using the Gemini API to identify Generalizations (GEN), Unfairness (UNFAIR), and Stereotypes (STEREO).

In [1]:
# Install dependencies if not already installed
!pip install -q google-generativeai datasets tqdm python-dotenv

In [1]:
import os
import json
import time
from tqdm import tqdm
import google.generativeai as genai
from datasets import load_dataset
from dotenv import load_dotenv

# --- CONFIGURATION ---
# Load API key from .env file
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

if not GOOGLE_API_KEY:
    raise ValueError("Please set GOOGLE_API_KEY in your .env file.")

genai.configure(api_key=GOOGLE_API_KEY)

# Model Name: User requested 'gemini 3 flash'. 
MODEL_NAME = "gemini-3-flash-preview"
model = genai.GenerativeModel(MODEL_NAME)

In [2]:
# --- DEFINITIONS & PROMPTS ---

SYSTEM_INSTRUCTION = """
You are an expert annotator for ethical AI datasets. Your task is to annotate text for three specific categories of bias: 
1. Generalizations (GEN)
2. Unfairness (UNFAIR)
3. Stereotypes (STEREO)

Please analyze the input text and provide annotations in BIO format tags (B-TAG, I-TAG, O) for each word/token in the text. 
However, to simplify the output for this specific task, please return the result as a JSON object containing the list of tags corresponding to the words in the input sentence.

### Definitions

**1. GEN (Generalizations)**
Tag: B-GEN / I-GEN
Description: Any broad generalization of a group or ubiquitous classifiers, including adjectives/descriptors (e.g., 'my whole family', 'all Muslims', 'Women in general', 'everyone in this city', 'all politicians', 'tall people', 'women', 'the rocketry club', 'students', 'tall boys')

**2. UNFAIR (Unfairness)**
Tag: B-UNFAIR / I-UNFAIR
Description: Any harsh or unjust characterization or offensive language. (e.g., 'such an idiot', 'stupid idea', 'totally useless', 'worst team ever', 'bad driver')

**3. STEREO (Stereotypes)**
Tag: B-STEREO / I-STEREO
Description: Any statement (multiple words) that contain a stereotype targeting a group of people, both explicitly and unconsiously. (e.g., 'women are bad drivers', 'Asians are good at math', 'Southerners are dumb', 'teenagers are irresponsible', 'men are gross').

### Output Format
Return a JSON object with a single key "annotations" which is a list of strings representing the BIO tags for the input tokens. 
Ensure the length of the tag list matches the number of words/tokens in the input.
Also include a brief "rationale" string explaining your decision.

Example Output JSON:
{
  "annotations": ["O", "B-GEN", "I-GEN", "O"],
  "rationale": "Explanation here..."
}
"""

GEN_examples = {
    "input_str": [
        "Women are all such bad drivers",
        "Everyone from the city is rude and unfriendly",
        "John is usually late to meetings",
        "The college students on my street are always so loud"
    ],
    "rationale": [
        "The use of the word 'women' in this broad statement is a generalization because it is not about any specific woman or group of women. The use of the words 'are' and 'all' is a ubiquitous classification, extending the generalization entity another two words.",
        "The phrase 'Everyone from the city' generalizes the behavior of all people from the city, which is a broad and unjustified assumption. The words 'is' and 'rude and unfriendly' extend this generalization.",
        "This phrase makes a statement about John's punctuality, but does it is specific to one person, and uses the word 'usually' which is a hedge and not a generalization.",
        "This phrase unfairly characterizes a large group of people (country people) as loud. The use of the word 'always' is a ubiquitious classifier, therefore this contains a generalization."
    ],
    "annotations": [
        ["B-GEN", "I-GEN", "I-GEN", "O", "O", "O"],
        ["B-GEN", "I-GEN", "I-GEN", "I-GEN", "O", "O", "O"],
        ["O", "O", "O", "O", "O", "O", "O"],
        ["O", "B-GEN", "I-GEN", "I-GEN", "I-GEN", "I-GEN", "I-GEN", "O", "O"]
    ]
}

# Constructing Few-Shot Examples for the Prompt
FEW_SHOT_PROMPT = ""
for i, inp in enumerate(GEN_examples["input_str"]):
    FEW_SHOT_PROMPT += f"Input: {inp}\n"
    FEW_SHOT_PROMPT += f"Rationale: {GEN_examples['rationale'][i]}\n"
    FEW_SHOT_PROMPT += f"Annotations: {GEN_examples['annotations'][i]}\n\n"


In [3]:
# --- LOAD DATASET (Hugging Face) ---
print("Loading HF dataset...")
dataset = load_dataset("ethical-spectacle/gus-dataset-v1", split="train") # Adjust split if needed
print(f"Loaded {len(dataset)} examples.")

# Extract text_str
texts_hf = dataset["text_str"]
# limitting to 5 for testing
test_texts = texts_hf[:5]
print(f"Sample text: {test_texts[0]}")

In [4]:
# --- ANNOTATION FUNCTION ---

def annotate_text(text):
    prompt = f"""{SYSTEM_INSTRUCTION}

Here are some examples:
{FEW_SHOT_PROMPT}

Now annotate this input:
Input: {text}
Provide the Output JSON:
"""
    
    try:
        response = model.generate_content(prompt, generation_config={"response_mime_type": "application/json"})
        return json.loads(response.text)
    except Exception as e:
        print(f"Error annotating text: {text[:50]}... | Error: {e}")
        return None


In [5]:
# --- RUN ANNOTATION LOOP (Hugging Face Dataset) ---

output_file_hf = "gemini_annotations.json"

# 1. Load existing annotations if available
if os.path.exists(output_file_hf):
    with open(output_file_hf, 'r') as f:
        try:
            annotated_results_hf = json.load(f)
            print(f"Loaded {len(annotated_results_hf)} existing annotations.")
        except json.JSONDecodeError:
            print("Could not decode existing JSON. Starting from scratch (backing up old file).")
            os.rename(output_file_hf, f"{output_file_hf}.bak")
            annotated_results_hf = []
else:
    annotated_results_hf = []

# 2. Determine start index
start_index = len(annotated_results_hf)
print(f"Resuming annotation from index {start_index} in HF dataset...")

# 3. Processing Loop
# Iterate through the remaining texts
for i, text in enumerate(tqdm(texts_hf[start_index:], desc="Annotating HF", initial=start_index, total=len(texts_hf))):
    
    # Retry logic for robustness
    max_retries = 3
    result = None
    for attempt in range(max_retries):
        result = annotate_text(text)
        if result:
            break
        print(f"Retry {attempt+1}/{max_retries}...")
        time.sleep(2) # Wait a bit longer on retry

    # Append result (success or failure)
    if result:
        annotated_results_hf.append({
            "text": text,
            "gemini_annotations": result.get("annotations", []),
            "gemini_rationale": result.get("rationale", "")
        })
    else:
        annotated_results_hf.append({
            "text": text,
            "gemini_annotations": [],
            "gemini_rationale": "ERROR_FAILED_AFTER_RETRIES"
        })

    # 4. Incremental Save (Every 10 items)
    if (len(annotated_results_hf)) % 10 == 0:
        with open(output_file_hf, 'w') as f:
            json.dump(annotated_results_hf, f, indent=2)

    # Rate limit sleep
    time.sleep(1.0) 

# Final Save
with open(output_file_hf, 'w') as f:
    json.dump(annotated_results_hf, f, indent=2)

print(f"Completed HF! Total annotations: {len(annotated_results_hf)} saved to {output_file_hf}")

## Local No-Bias Dataset Annotation

In [6]:
# --- LOAD LOCAL DATASET ---
import json

print("Loading Local dataset from '../dataset/bias_sentences.json'...")
try:
    with open("../dataset/bias_sentences.json", "r") as f:
        data_local = json.load(f)

    # Filter for no bias
    entries_no_bias = [entry for entry in data_local["entries"] if not entry.get("has_bias", True)]
    texts_local = [entry["text"] for entry in entries_no_bias]

    print(f"Loaded {len(texts_local)} examples without bias.")
    if texts_local:
        print(f"Sample text: {texts_local[0]}")
except FileNotFoundError:
    print("Error: '../dataset/bias_sentences.json' not found. Please ensure the file exists.")
    texts_local = []

In [7]:
# --- RUN ANNOTATION LOOP (Local No-Bias Dataset) ---

output_file_local = "no_bias_annotations.json"

# 1. Load existing annotations if available
if os.path.exists(output_file_local):
    with open(output_file_local, 'r') as f:
        try:
            annotated_results_local = json.load(f)
            print(f"Loaded {len(annotated_results_local)} existing annotations.")
        except json.JSONDecodeError:
            print("Could not decode existing JSON. Starting from scratch.")
            os.rename(output_file_local, f"{output_file_local}.bak")
            annotated_results_local = []
else:
    annotated_results_local = []

# 2. Determine start index
start_index_local = len(annotated_results_local)
print(f"Resuming annotation from index {start_index_local} in Local dataset...")

# 3. Processing Loop
if texts_local:
    for i, text in enumerate(tqdm(texts_local[start_index_local:], desc="Annotating Local", initial=start_index_local, total=len(texts_local))):
        
        max_retries = 3
        result = None
        for attempt in range(max_retries):
            result = annotate_text(text)
            if result:
                break
            print(f"Retry {attempt+1}/{max_retries}...")
            time.sleep(2)

        if result:
            annotated_results_local.append({
                "text": text,
                "gemini_annotations": result.get("annotations", []),
                "gemini_rationale": result.get("rationale", "")
            })
        else:
            annotated_results_local.append({
                "text": text,
                "gemini_annotations": [],
                "gemini_rationale": "ERROR_FAILED_AFTER_RETRIES"
            })

        if (len(annotated_results_local)) % 10 == 0:
            with open(output_file_local, 'w') as f:
                json.dump(annotated_results_local, f, indent=2)

        time.sleep(1.0)

    with open(output_file_local, 'w') as f:
        json.dump(annotated_results_local, f, indent=2)

    print(f"Completed Local! Total annotations: {len(annotated_results_local)} saved to {output_file_local}")
else:
    print("Skipping local annotation loop due to missing data.")